In [ ]:
from requests_html import HTMLSession
# import scrapy
# from scrapy.crawler import CrawlerProcess
# import pandas as pd
# import sqlite3
from trafilatura import extract , fetch_url
from pprint import pprint as print

In [ ]:
def data_next_page_id(base_size=60, maximum_size=110):
    count = 1
    session = HTMLSession()
    next_page_ids = [0]
    while count != 0:
        new_next_page_ids = []
        for _ in next_page_ids:
            url = f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size={base_size}-{maximum_size}'
            response = session.get(url)
            new_next_page_ids.append(response.json().get("next_page_id"))
        next_page_ids.extend(new_next_page_ids)  # Use extend() instead of append()
        count -= 1
    return list(set(next_page_ids))

In [ ]:
apartment = data_next_page_id(60,110)
def tokens():
    urls=[]
    tokens=[]
    for _ in apartment:
        urls.append(f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size=60-110')
    for url in urls:
        session = HTMLSession()
        response=session.get(url)
        datas=response.json()["posts"]
        for _ in datas:
            tokens.append(_.get("token"))
    return tokens
# tokens()

In [ ]:
urls=[f"https://divar.ir/v/{_}" for _ in tokens()]

urls